In [1]:
import re
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import geopandas as gpd
from shapely.geometry import Point, LineString
import networkx as nx
import gurobipy as gb
from keplergl import KeplerGl
from gtfs_functions import Feed
from sklearn.metrics.pairwise import manhattan_distances

In [2]:
logging.getLogger().setLevel(logging.WARNING)

## Loading data


In [ ]:
NUM_ROUTES = 3
distance_matrix = pd.read_json("distance_matrix.json")

In [ ]:
# add depot to distance matrix
for i in range(NUM_ROUTES):
    distance_matrix.loc[i+1] = 0.0
    distance_matrix[i+1] = 0.0

distance_matrix

In [ ]:
np.nanmax(distance_matrix.to_numpy())

In [ ]:
distance_matrix.fillna(1e2, inplace=True)

In [ ]:
distance_matrix

In [ ]:
nodes = list(distance_matrix.index)
num_nodes = len(nodes)

# TSP - Bus Route Optimization

In [ ]:
vrp = gb.Model("Montreal Bus Routing")
vrp.Params.MIPGap = 0.3
vrp.Params.TimeLimit = 30
start_node_index = 68


# Decision variables
x = vrp.addVars(nodes, nodes, vtype=gb.GRB.BINARY, name="x")

u = vrp.addVars(nodes, vtype=gb.GRB.INTEGER, name="u", ub=num_nodes)

# Objective function
vrp.setObjective(
    gb.quicksum(distance_matrix.loc[i, j] * x[i, j] for i in nodes for j in nodes),
    gb.GRB.MINIMIZE,
)

# Constraints
vrp.addConstrs(
    (gb.quicksum(x[i, j] for j in nodes if j != i) == 1 for i in nodes),
    name="outgoing",
)

vrp.addConstrs(
    (gb.quicksum(x[j, i] for j in nodes if j != i) == 1 for i in nodes),
    name="ingoing",
)

vrp.addConstrs(
    (
        u[i] - u[j] + num_nodes * x[i, j] <= num_nodes - 1
        for i in nodes
        for j in nodes
        if i != j and i != nodes[start_node_index] and j != nodes[start_node_index]
    ),
    name="subtour_elimination",
)

vrp.addConstrs(
    (u[i] >= 2 for i in nodes if i != nodes[start_node_index]), name="lower bound for u"
)

vrp.addConstr(
    u[nodes[start_node_index]] == 1,
    name="start node index is 1",
)

vrp.optimize()

In [ ]:
nodes[start_node_index]

In [ ]:
print("Minimum distance: ", vrp.objVal)
print("Optimal path: ")
tour = []
for v in vrp.getVars():
    if v.varName.startswith("u"):
        tour.append(v)

for s in sorted(tour, key=lambda x: x.x):
    print(s.varName, s.x)

In [ ]:
# Distances
distances = []

for i in nodes:
    for j in nodes:
        if x[i, j].x == 1:
            distances.append(distance_matrix[i][j])

pd.Series(distances).describe()

In [ ]:
# Plotting the directed graph
G = nx.DiGraph()

for i in nodes:
    for j in nodes:
        if x[i, j].x == 1:
            G.add_edge(
                i, j, weight=distance_matrix[i][j], step=u[i].x, label=f"{i}-{j}"
            )

pos = nx.spring_layout(G)
steps = nx.get_edge_attributes(G, "step")

from pyvis.network import Network

net = Network(
    directed=True,
    notebook=True,
    select_menu=True,
    filter_menu=True,
    cdn_resources="remote",
    neighborhood_highlight=True,
)

net.from_nx(G)
net.show_buttons()


net.show("example.html")

In [ ]:
# convert tour into dictionary of steps and nodes

tour_dict = {}

for node in tour:
    node_name = node.varName

    node_name = re.sub(r"[u\[\]]", "", node_name)
    tour_dict[node.x] = int(node_name)

tour_df = (
    pd.DataFrame.from_dict(tour_dict, orient="index", columns=["node"])
    .sort_index()
    .reset_index()
    .rename(columns={"index": "step"})
)

tour_df = tour_df.merge(stops, left_on="node", right_on="stop_code", how="left")

tour_df["stop_lat"] = tour_df["stop_lat"].fillna(45.5376881)
tour_df["stop_lon"] = tour_df["stop_lon"].fillna(-73.5705049)
tour_df["stop_name"] = tour_df["stop_name"].fillna("Depot")

tour_df["geometry"] = tour_df.apply(
    lambda x: Point((float(x.stop_lon), float(x.stop_lat))), axis=1
)

tour_df = gpd.GeoDataFrame(tour_df, geometry="geometry")

tour_df

In [ ]:
depot_steps = tour_df[tour_df["stop_name"] == "Depot"].index.tolist()

tours = []

for i in range(len(depot_steps) - 1):
    tours.append(tour_df.iloc[depot_steps[i] : depot_steps[i + 1] + 1])

tours.append(tour_df.iloc[depot_steps[-1] :])

In [ ]:
tour_lines = []

for i, tour_ in enumerate(tours):
    tour_lines.append({"route_id": i, "geometry": LineString(tour_.geometry.tolist())})


tour_lines = gpd.GeoDataFrame.from_dict(tour_lines, geometry="geometry")

tour_lines

In [ ]:
config = {
    "version": "v1",
    "config": {
        "visState": {
            "filters": [
                {
                    "dataId": ["routes"],
                    "id": "cc52bgyolv",
                    "name": ["route_id"],
                    "type": "range",
                    "value": [0, 0.73],
                    "enlarged": False,
                    "plotType": "histogram",
                    "animationWindow": "free",
                    "yAxis": None,
                    "speed": 1,
                }
            ],
            "layers": [
                {
                    "id": "w9xelah",
                    "type": "point",
                    "config": {
                        "dataId": "stops_in_trip",
                        "label": "stop",
                        "color": [221, 178, 124],
                        "highlightColor": [252, 242, 26, 255],
                        "columns": {
                            "lat": "stop_lat",
                            "lng": "stop_lon",
                            "altitude": "step",
                        },
                        "isVisible": True,
                        "visConfig": {
                            "radius": 10,
                            "fixedRadius": False,
                            "opacity": 0.8,
                            "outline": False,
                            "thickness": 2,
                            "strokeColor": None,
                            "colorRange": {
                                "name": "Global Warming",
                                "type": "sequential",
                                "category": "Uber",
                                "colors": [
                                    "#5A1846",
                                    "#900C3F",
                                    "#C70039",
                                    "#E3611C",
                                    "#F1920E",
                                    "#FFC300",
                                ],
                            },
                            "strokeColorRange": {
                                "name": "Global Warming",
                                "type": "sequential",
                                "category": "Uber",
                                "colors": [
                                    "#5A1846",
                                    "#900C3F",
                                    "#C70039",
                                    "#E3611C",
                                    "#F1920E",
                                    "#FFC300",
                                ],
                            },
                            "radiusRange": [0, 50],
                            "filled": True,
                        },
                        "hidden": False,
                        "textLabel": [
                            {
                                "field": None,
                                "color": [255, 255, 255],
                                "size": 18,
                                "offset": [0, 0],
                                "anchor": "start",
                                "alignment": "center",
                            }
                        ],
                    },
                    "visualChannels": {
                        "colorField": {"name": "step", "type": "integer"},
                        "colorScale": "quantile",
                        "strokeColorField": None,
                        "strokeColorScale": "quantile",
                        "sizeField": None,
                        "sizeScale": "linear",
                    },
                },
                {
                    "id": "ez5bm5",
                    "type": "geojson",
                    "config": {
                        "dataId": "routes",
                        "label": "routes",
                        "color": [30, 150, 190],
                        "highlightColor": [252, 242, 26, 255],
                        "columns": {"geojson": "geometry"},
                        "isVisible": True,
                        "visConfig": {
                            "opacity": 0.8,
                            "strokeOpacity": 0.8,
                            "thickness": 1,
                            "strokeColor": None,
                            "colorRange": {
                                "name": "Global Warming",
                                "type": "sequential",
                                "category": "Uber",
                                "colors": [
                                    "#5A1846",
                                    "#900C3F",
                                    "#C70039",
                                    "#E3611C",
                                    "#F1920E",
                                    "#FFC300",
                                ],
                            },
                            "strokeColorRange": {
                                "name": "ColorBrewer Paired-5",
                                "type": "qualitative",
                                "category": "ColorBrewer",
                                "colors": [
                                    "#a6cee3",
                                    "#1f78b4",
                                    "#b2df8a",
                                    "#33a02c",
                                    "#fb9a99",
                                ],
                            },
                            "radius": 10,
                            "sizeRange": [0, 10],
                            "radiusRange": [0, 50],
                            "heightRange": [0, 500],
                            "elevationScale": 5,
                            "enableElevationZoomFactor": True,
                            "stroked": True,
                            "filled": False,
                            "enable3d": False,
                            "wireframe": False,
                        },
                        "hidden": False,
                        "textLabel": [
                            {
                                "field": None,
                                "color": [255, 255, 255],
                                "size": 18,
                                "offset": [0, 0],
                                "anchor": "start",
                                "alignment": "center",
                            }
                        ],
                    },
                    "visualChannels": {
                        "colorField": None,
                        "colorScale": "quantile",
                        "strokeColorField": {"name": "route_id", "type": "integer"},
                        "strokeColorScale": "quantile",
                        "sizeField": None,
                        "sizeScale": "linear",
                        "heightField": None,
                        "heightScale": "linear",
                        "radiusField": None,
                        "radiusScale": "linear",
                    },
                },
            ],
            "interactionConfig": {
                "tooltip": {
                    "fieldsToShow": {
                        "stops_in_trip": [
                            {"name": "stop_name", "format": None},
                            {"name": "step", "format": None},
                        ],
                        "routes": [{"name": "route_id", "format": None}],
                    },
                    "compareMode": False,
                    "compareType": "absolute",
                    "enabled": True,
                },
                "brush": {"size": 0.5, "enabled": False},
                "geocoder": {"enabled": False},
                "coordinate": {"enabled": False},
            },
            "layerBlending": "normal",
            "splitMaps": [],
            "animationConfig": {"currentTime": None, "speed": 1},
        },
        "mapState": {
            "bearing": 0,
            "dragRotate": False,
            "latitude": 45.59783403161273,
            "longitude": -73.62435981591366,
            "pitch": 0,
            "zoom": 10,
            "isSplit": False,
        },
        "mapStyle": {
            "styleType": "light",
            "topLayerGroups": {},
            "visibleLayerGroups": {
                "label": True,
                "road": True,
                "border": False,
                "building": True,
                "water": True,
                "land": True,
                "3d building": False,
            },
            "threeDBuildingColor": [
                218.82023004728686,
                223.47597962276103,
                223.47597962276103,
            ],
            "mapStyles": {},
        },
    },
}

In [ ]:
map = KeplerGl(height=700, config=config)

map.add_data(
    data=tour_df[["stop_lat", "stop_lon", "stop_name", "step"]],
    name="stops_in_trip",
)

map.add_data(
    data=tour_lines[['route_id', 'geometry']],
    name="routes",
)

map

# VRP - Number of buses

In [ ]:
# feed = Feed("./STM GTFS/gtfs_stm.zip", busiest_date=False)

In [ ]:
# routes = feed.routes
# trips = feed.trips
# stops = feed.stops
# stop_times = feed.stop_times
# shapes = feed.shapes
# segments = feed.segments

In [36]:
segments = pd.read_csv("segments.csv")
stops = pd.read_csv("stops.csv")

segments[['route_id', 'start_stop_id', 'end_stop_id']] = segments[['route_id', 'start_stop_id', 'end_stop_id']].astype(str)

In [37]:
def get_segment_stops(segment_df, route_id, direction_id=0):
    segment_distance = segment_df.query(
        "route_id == @route_id & direction_id == @direction_id"
    )

    stops_in_route = stops[
        stops["stop_id"].isin(
            list(
                set(
                    segment_distance[["start_stop_id", "end_stop_id"]].values.reshape(
                        -1
                    )
                )
            )
        )
    ]

    return segment_distance, stops_in_route

### Get stops on route

In [38]:
routes = ["35", "61", "15", "51", "18", "107", "24", "67"]
route_details = {}

for r in routes:
    detail = get_segment_stops(segments, r, 0)

    route_details[r] = {}
    route_details[r]["segment"] = detail[0]
    route_details[r]["stops"] = detail[1]

In [39]:
stops_df = pd.concat([v["stops"] for k, v in route_details.items()]).drop_duplicates()
print(f"Number of total stops in routes: {len(stops_df)}")
stops_df.head()

Number of total stops in routes: 295


,stop_id,stop_code,stop_name,stop_lat,stop_lon,stop_url,location_type,parent_station,wheelchair_boarding,geometry
422,61743,61743,Notre-Dame / Guy,45.489959,-73.567233,https://www.stm.info/fr/recherche#stq=61743,0,NaN,1,POINT (-73.567233 45.489959)
423,61744,61744,de la Montagne / Ottawa,45.492014,-73.561145,https://www.stm.info/fr/recherche#stq=61744,0,NaN,1,POINT (-73.561145 45.492014)
424,61745,61745,de la Montagne / du Square-Gallery,45.491756,-73.558727,https://www.stm.info/fr/recherche#stq=61745,0,NaN,1,POINT (-73.558727 45.491756)
438,61765,61765,Wellington / Prince,45.496656,-73.555554,https://www.stm.info/fr/recherche#stq=61765,0,NaN,1,POINT (-73.555554 45.496656)
591,62063,62063,Notre-Dame / Bérard,45.480191,-73.579727,https://www.stm.info/fr/recherche#stq=62063,0,NaN,1,POINT (-73.579727 45.480191)


## Visualize stops on a map

In [40]:
stops_df["geometry"] = stops_df.apply(
    lambda x: Point((float(x.stop_lon), float(x.stop_lat))), axis=1
)

stops_df_gpd = gpd.GeoDataFrame(stops_df, geometry="geometry").drop(
    columns=["location_type", "parent_station", "wheelchair_boarding"]
)

stops_df_gpd.head()

,stop_id,stop_code,stop_name,stop_lat,stop_lon,stop_url,geometry
422,61743,61743,Notre-Dame / Guy,45.489959,-73.567233,https://www.stm.info/fr/recherche#stq=61743,POINT (-73.56723 45.48996)
423,61744,61744,de la Montagne / Ottawa,45.492014,-73.561145,https://www.stm.info/fr/recherche#stq=61744,POINT (-73.56114 45.49201)
424,61745,61745,de la Montagne / du Square-Gallery,45.491756,-73.558727,https://www.stm.info/fr/recherche#stq=61745,POINT (-73.55873 45.49176)
438,61765,61765,Wellington / Prince,45.496656,-73.555554,https://www.stm.info/fr/recherche#stq=61765,POINT (-73.55555 45.49666)
591,62063,62063,Notre-Dame / Bérard,45.480191,-73.579727,https://www.stm.info/fr/recherche#stq=62063,POINT (-73.57973 45.48019)


In [52]:
random_stops_df_gpd = stops_df_gpd.sample(frac=0.3, random_state=420)

random_stops_df_gpd.head()

,stop_id,stop_code,stop_name,stop_lat,stop_lon,stop_url,geometry
4279,54019,54019,Sherbrooke / De Champlain (Hôpital Notre-Dame),45.526576,-73.564249,https://www.stm.info/fr/recherche#stq=54019,POINT (-73.56425 45.52658)
2467,52009,52009,Sherbrooke / Chomedey,45.492614,-73.585807,https://www.stm.info/fr/recherche#stq=52009,POINT (-73.58581 45.49261)
2130,51629,51629,Beaubien / De Lanaudière,45.540561,-73.598947,https://www.stm.info/fr/recherche#stq=51629,POINT (-73.59895 45.54056)
3356,53011,53011,Hochelaga / Davidson,45.545120,-73.552587,https://www.stm.info/fr/recherche#stq=53011,POINT (-73.55259 45.54512)
6976,56573,56573,de Verdun / Manning,45.449240,-73.572503,https://www.stm.info/fr/recherche#stq=56573,POINT (-73.57250 45.44924)


In [53]:
# Add a depot stop to the random stops dataframe in the beginning
depot = pd.DataFrame(
    {
        "stop_id": ["0"],
        "stop_name": ["Depot"],
        "stop_lat": [45.5048542],
        "stop_lon": [-73.5691235],
        "stop_code": 0.0,
    }
)

depot["geometry"] = Point((float(depot.stop_lon), float(depot.stop_lat)))

random_stops_df_gpd = pd.concat([depot, random_stops_df_gpd]).reset_index(drop=True)

random_stops_df_gpd

,stop_id,stop_name,stop_lat,stop_lon,stop_code,geometry,stop_url
0,0,Depot,45.504854,-73.569124,0.0,POINT (-73.5691235 45.5048542),NaN
1,54019,Sherbrooke / De Champlain (Hôpital Notre-Dame),45.526576,-73.564249,54019.0,POINT (-73.564249 45.526576),https://www.stm.info/fr/recherche#stq=54019
2,52009,Sherbrooke / Chomedey,45.492614,-73.585807,52009.0,POINT (-73.585807 45.492614),https://www.stm.info/fr/recherche#stq=52009
3,51629,Beaubien / De Lanaudière,45.540561,-73.598947,51629.0,POINT (-73.598947 45.540561),https://www.stm.info/fr/recherche#stq=51629
4,53011,Hochelaga / Davidson,45.545120,-73.552587,53011.0,POINT (-73.552587 45.54512),https://www.stm.info/fr/recherche#stq=53011
...,...,...,...,...,...,...,...
84,56211,Laurier / Querbes,45.517311,-73.597705,56211.0,POINT (-73.597705 45.517311),https://www.stm.info/fr/recherche#stq=56211
85,50576,West Broadway / Fielding,45.459892,-73.648730,50576.0,POINT (-73.64873 45.459892),https://www.stm.info/fr/recherche#stq=50576
86,52445,Beaubien / 42e Avenue,45.574052,-73.569634,52445.0,POINT (-73.569634 45.574052),https://www.stm.info/fr/recherche#stq=52445
87,56627,Wellington / 1re Avenue,45.458319,-73.567345,56627.0,POINT (-73.567345 45.458319),https://www.stm.info/fr/recherche#stq=56627


### Calculate Manhattan distance between stops

In [54]:
distance_matrix = pd.DataFrame(
    manhattan_distances(random_stops_df_gpd[["stop_lat", "stop_lon"]].values),
    index=random_stops_df_gpd["stop_id"],
    columns=random_stops_df_gpd["stop_id"],
)

display(distance_matrix.head())

stop_id,0,54019,52009,51629,53011,56573,51856,51496,53898,56631,...,52795,52505,56600,53003,50931,56211,50576,52445,56627,51743
stop_id,,,,,,,,,,,,,,,,,,,,,
0,0.000000,0.026596,0.028924,0.065530,0.056802,0.058994,0.039731,0.067042,0.031088,0.045169,...,0.095242,0.029943,0.037713,0.031632,0.129248,0.041038,0.124569,0.069708,0.048314,0.077123
54019,0.026596,0.000000,0.055520,0.048683,0.030206,0.085590,0.028486,0.093638,0.047935,0.067582,...,0.068646,0.056539,0.064309,0.016072,0.112401,0.042721,0.151165,0.052861,0.071353,0.060276
52009,0.028924,0.055520,0.000000,0.061087,0.085726,0.056678,0.035288,0.038118,0.035531,0.049612,...,0.124166,0.031761,0.038921,0.060556,0.124805,0.036595,0.095645,0.097611,0.052757,0.072680
51629,0.065530,0.048683,0.061087,0.000000,0.050919,0.117765,0.025799,0.072925,0.096618,0.110699,...,0.089359,0.092848,0.100008,0.064755,0.063718,0.024492,0.130452,0.062804,0.113844,0.022065
53011,0.056802,0.030206,0.085726,0.050919,0.000000,0.115796,0.058692,0.123844,0.064879,0.097788,...,0.051812,0.086745,0.094515,0.025170,0.105519,0.072927,0.181371,0.045979,0.101559,0.053394


In [12]:
config = {
    "version": "v1",
    "config": {
        "visState": {
            "filters": [],
            "layers": [
                {
                    "id": "pha44gc",
                    "type": "point",
                    "config": {
                        "dataId": "stops",
                        "label": "stop",
                        "color": [30, 150, 190],
                        "highlightColor": [252, 242, 26, 255],
                        "columns": {
                            "lat": "stop_lat",
                            "lng": "stop_lon",
                            "altitude": None,
                        },
                        "isVisible": True,
                        "visConfig": {
                            "radius": 15,
                            "fixedRadius": False,
                            "opacity": 1,
                            "outline": False,
                            "thickness": 2,
                            "strokeColor": [28, 27, 27],
                            "colorRange": {
                                "name": "Uber Viz Qualitative 3",
                                "type": "qualitative",
                                "category": "Uber",
                                "colors": [
                                    "#12939A",
                                    "#DDB27C",
                                    "#88572C",
                                    "#FF991F",
                                    "#F15C17",
                                    "#223F9A",
                                    "#DA70BF",
                                    "#125C77",
                                    "#4DC19C",
                                    "#776E57",
                                    "#17B8BE",
                                    "#F6D18A",
                                    "#B7885E",
                                    "#FFCB99",
                                    "#F89570",
                                ],
                            },
                            "strokeColorRange": {
                                "name": "Global Warming",
                                "type": "sequential",
                                "category": "Uber",
                                "colors": [
                                    "#5A1846",
                                    "#900C3F",
                                    "#C70039",
                                    "#E3611C",
                                    "#F1920E",
                                    "#FFC300",
                                ],
                            },
                            "radiusRange": [0, 50],
                            "filled": True,
                        },
                        "hidden": False,
                        "textLabel": [
                            {
                                "field": None,
                                "color": [255, 255, 255],
                                "size": 18,
                                "offset": [0, 0],
                                "anchor": "start",
                                "alignment": "center",
                            }
                        ],
                    },
                    "visualChannels": {
                        "colorField": {"name": "stop_name", "type": "string"},
                        "colorScale": "ordinal",
                        "strokeColorField": None,
                        "strokeColorScale": "quantile",
                        "sizeField": None,
                        "sizeScale": "linear",
                    },
                }
            ],
            "interactionConfig": {
                "tooltip": {
                    "fieldsToShow": {
                        "stops": [
                            {"name": "stop_name", "format": None},
                            {"name": "stop_id", "format": None},
                        ]
                    },
                    "compareMode": False,
                    "compareType": "absolute",
                    "enabled": True,
                },
                "brush": {"size": 0.5, "enabled": False},
                "geocoder": {"enabled": False},
                "coordinate": {"enabled": True},
            },
            "layerBlending": "normal",
            "splitMaps": [],
            "animationConfig": {"currentTime": None, "speed": 1},
        },
        "mapState": {
            "bearing": 0,
            "dragRotate": False,
            "latitude": 45.49373589563174,
            "longitude": -73.57232342042659,
            "pitch": 0,
            "zoom": 11.401161292661241,
            "isSplit": False,
        },
        "mapStyle": {
            "styleType": "light",
            "topLayerGroups": {},
            "visibleLayerGroups": {
                "label": True,
                "road": True,
                "border": False,
                "building": True,
                "water": True,
                "land": True,
                "3d building": False,
            },
            "threeDBuildingColor": [
                218.82023004728686,
                223.47597962276103,
                223.47597962276103,
            ],
            "mapStyles": {},
        },
    },
}

In [55]:
stop_map = KeplerGl(height=700, config=config)

stop_map.add_data(
    data=random_stops_df_gpd[["stop_lat", "stop_lon", "stop_name", "stop_id"]],
    name="stops",
)

stop_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'ewggvqp', 'type': …

## Using Gurobi to solve the VRP - no capacity constraints

In [56]:
# --------------------------------------------------------------------------------
# Paramters
distance_matrix_model = distance_matrix

stops = list(distance_matrix_model.index)
num_stops = len(stops)

num_buses = 5

model = gb.Model("Bus Routing")
model.Params.MIPGap = 0.1
model.Params.TimeLimit = 60

# display(distance_matrix_model)

# --------------------------------------------------------------------------------
# Decision variables
x = model.addVars(
    stops,
    stops,
    num_buses,
    vtype=gb.GRB.BINARY,
    name=(
        f"{i} -> {j} (Bus {k+1})"
        for i in stops
        for j in stops
        for k in range(num_buses)
    ),
)

u = model.addVars(
    stops,
    num_buses,
    vtype=gb.GRB.INTEGER,
    lb=0,
    ub=num_stops,
    name=(f"Step for stop {i} (Bus {k+1})" for i in stops for k in range(num_buses)),
)

# --------------------------------------------------------------------------------
# Objective function
model.setObjective(
    gb.quicksum(
        distance_matrix_model.loc[i, j] * x[i, j, k]
        for i in stops
        for j in stops
        for k in range(num_buses)
    ),
    gb.GRB.MINIMIZE,
)

# --------------------------------------------------------------------------------
# Constraints

# Each stop is visited once by one vehicle only
model.addConstrs(
    (
        gb.quicksum(x[i, j, k] for j in stops if j != i for k in range(num_buses)) == 1
        for i in stops[1:]
    ),
    name="outgoing",
)

model.addConstrs(
    (
        gb.quicksum(x[i, j, k] for i in stops if i != j for k in range(num_buses)) == 1
        for j in stops[1:]
    ),
    name="incoming",
)

# Each vehicle leaves the depot and returns to the depot
model.addConstrs(
    (
        gb.quicksum(x[stops[0], j, k] for j in stops if j != stops[0]) == 1
        for k in range(num_buses)
    ),
    name="Vehicle leaves depot",
)

model.addConstrs(
    (
        gb.quicksum(x[i, stops[0], k] for i in stops if i != stops[0]) == 1
        for k in range(num_buses)
    ),
    name="Vehicle returns to depot",
)

# Flow conservation (routes are continuous)
model.addConstrs(
    (
        gb.quicksum(x[j, i, k] for j in stops if j != i)
        - gb.quicksum(x[i, j, k] for j in stops if j != i)
        == 0
        for i in stops[1:]
        for k in range(num_buses)
    ),
    name="Flow conservation",
)

# Subtour elimination
model.addConstrs(
    (
        u[i, k] - u[j, k] + num_stops * x[i, j, k] <= num_stops - 1
        for i in stops
        for j in stops
        for k in range(num_buses)
        if i != j and i != stops[0] and j != stops[0]
    ),
    name="Subtour elimination",
)

model.addConstrs(
    (u[stops[0], k] == 0 for k in range(num_buses)), name="Start node index is 0 (depot) for all buses"
)

Set parameter MIPGap to value 0.1


Set parameter TimeLimit to value 60


{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>}

In [57]:
# Optimize
model.update()
model.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 38911 rows, 40050 columns and 270605 nonzeros
Model fingerprint: 0xb575d538
Variable types: 0 continuous, 40050 integer (39605 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [1e-03, 2e-01]
  Bounds range     [1e+00, 9e+01]
  RHS range        [1e+00, 9e+01]
Presolve removed 5 rows and 450 columns
Presolve time: 0.51s
Presolved: 38906 rows, 39600 columns, 270600 nonzeros
Variable types: 0 continuous, 39600 integer (39160 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Use crossover to convert LP symmetric solution to basic solution...

Root relaxation: objective 6.093062e-01, 668 iterations, 0.22 seconds (0.

In [58]:
def print_optimal_paths(model, num_buses, stops):
    routes = {}

    print("Minimum distance: ", model.objVal)
    print("Optimal path for each bus: ")

    for k in range(num_buses):
        print(f"Bus {k+1}:")

        # Initialize with the depot as the starting point
        current_stop = stops[0]
        next_stop = None
        route = [current_stop]  # Start with the depot

        # Follow the path for the current bus
        while next_stop != stops[0]:
            for j in stops:
                if j != current_stop and x[current_stop, j, k].x > 0.5:
                    next_stop = j
                    route.append(next_stop)
                    current_stop = next_stop
                    break
            else:
                # If no next stop is found and we are not at the depot, it indicates an issue
                if current_stop != stops[0]:
                    print(
                        "Error: No valid next stop found. Check the model constraints."
                    )
                break

        routes[k] = route
        print(" -> ".join(map(str, route)))

    return routes

In [59]:
optimal_routes = print_optimal_paths(model, num_buses, stops)

Minimum distance:  1.1380063999999805
Optimal path for each bus: 
Bus 1:
0 -> 52543 -> 53003 -> 53047 -> 53011 -> 52431 -> 52520 -> 0
Bus 2:
0 -> 52434 -> 0
Bus 3:
0 -> 53843 -> 52176 -> 52335 -> 0
Bus 4:
0 -> 52498 -> 0
Bus 5:
0 -> 52488 -> 52510 -> 52647 -> 54019 -> 52701 -> 52720 -> 52673 -> 52840 -> 52795 -> 52414 -> 52445 -> 54207 -> 50101 -> 55334 -> 50732 -> 50814 -> 50931 -> 51080 -> 51142 -> 51517 -> 51606 -> 51743 -> 52046 -> 60634 -> 51983 -> 51869 -> 54136 -> 51741 -> 51629 -> 51551 -> 53851 -> 51482 -> 54398 -> 51856 -> 56211 -> 56196 -> 56184 -> 51388 -> 51287 -> 51214 -> 51241 -> 51032 -> 56328 -> 56324 -> 50576 -> 50799 -> 51496 -> 52123 -> 52053 -> 52505 -> 56600 -> 56597 -> 56593 -> 56685 -> 56552 -> 56573 -> 56577 -> 56706 -> 56631 -> 56636 -> 56627 -> 51951 -> 52009 -> 52069 -> 52066 -> 52325 -> 52482 -> 61743 -> 59235 -> 52717 -> 62165 -> 52912 -> 53898 -> 59223 -> 58803 -> 52666 -> 52705 -> 0


## Visualize routes to be taken by buses

In [60]:
# For each optimal route, generate a GeoDataFrame with the stops and the route


def get_optimal_route_gdf(optimal_routes, stops_df_gpd):
    routes_gdf = {}

    for k, route in optimal_routes.items():
        # Add stops as GeoDataFrame with steps as index of route
        for step, stop in enumerate(route):
            stop_gdf = stops_df_gpd[stops_df_gpd["stop_id"] == stop]
            stop_gdf["step"] = step
            routes_gdf[k] = routes_gdf.get(k, pd.DataFrame()).append(stop_gdf)

    return routes_gdf


routes_gdf = get_optimal_route_gdf(optimal_routes, random_stops_df_gpd)

# add linestring of each point to the next point in the route


def get_route_lines(routes_gdf):
    route_lines = {}

    for k, route in routes_gdf.items():
        route_lines[k] = {
            "route_id": k,
            "geometry": LineString(route.geometry.tolist()),
        }

    return route_lines


route_lines = gpd.GeoDataFrame(
    pd.DataFrame(get_route_lines(routes_gdf)).T.reset_index(drop=True),
    geometry="geometry",
)

In [61]:
route_map = KeplerGl(height=900, config=config)

route_map.add_data(
    data=route_lines[["route_id", "geometry"]],
    name="routes",
)

for k, route in routes_gdf.items():
    route_map.add_data(
        data=route[["stop_lat", "stop_lon", "stop_id", "stop_name", "step"]],
        name=f"stops_in_trip_{k}",
    )

route_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'ewggvqp', 'type': …